# Introduction

This notebook demonstrates how to use partiiton strategy to solve larger scale route optimization problem. The rationale for partitioning is that usually an optimization problem could be hard to solve given the NP-hard nature for most of the optimization problems. To trade-off the result optimality and running time, one can partition the big problem into many smaller problems, then solve each smaller problem, and finially combine all results as the final result. The whole pipeline is illustrated by the below figure.

<img src=../docs/media/pipeline.png width="90%" />

There are 4 main steps in the pipeline:
1.  Reduce: It will try to assign some of the orders to truck routes in a heuristic way. The remaining unscheduled order will be passed to the later steps for optimization. This step is optional, namely, one can bypass this step but let optimizer search solution for all orders. However, reducing the search space by  heuristic can significantly reduce the search space. This will make it easier for the oprimization solver to find a good solution.  
2.  Partition: This is core step to partition the big problem into smaller problems. 
3.  Solve: This step is to solve individual small problem using whatever optimization solver.
4.  Merge: This final step is to combine all results from each small problem.




# 1.0 Load libraries

We use Azure ML pipeline for the implementation. Specifically, the partitioning step is done by the PrallelRunStep in Azure ML SDK.

In [1]:
# import required libraries
import os
from dotenv import load_dotenv

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input, Output, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Environment
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml.parallel import parallel_run_function, RunFunction

# 1.1 Setup some environment
## 1.1.1 Load variables

Some parameters are managed by environment variables.To specify your values, create a .env file in the root folder of the repository and set the values for the following parameters.

In [ ]:
load_dotenv()

ws_name = os.environ['AML_WORKSPACE_NAME']
subscription_id = os.environ['AML_SUBSCRIPTION_ID']
resource_group = os.environ['AML_RESOURCE_GROUP']


print('---- Check Azure setting ----')
print(f'AML Workspace name       : {ws_name}')
print(f'Subscription ID          : {subscription_id}')
print(f'Resource group           : {resource_group}')

## 1.1.2 Azure authentication and Load Azure ML Workspace

We are using DefaultAzureCredential to get access to workspace.

DefaultAzureCredential should be capable of handling most Azure SDK authentication scenarios.

Reference for more available credentials if it does not work for you: configure credential example, azure-identity reference doc.

In [3]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": subscription_id,
        "resource_group": resource_group,
        "workspace_name": ws_name,
    }

    if client_config["subscription_id"].startswith("<"):
        print(
            "please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <AML_WORKSPACE_NAME> in notebook cell"
        )
        raise ex
    else:  # write and reload from config file
        import json, os

        config_path = "../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

## 1.1.3 Get Compute Cluster

Read the compute name from the environment varibale. If it doest not exist in the Azure ML workspace, a new compute target will be created.

In [5]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name for the optimization job
cpu_compute_target = "op-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="Standard_E4ds_v4", min_instances=0, max_instances=10
    )
    ml_client.compute.begin_create_or_update(compute).result()

## 1.1.4 Create AML Environemnt and Run Configuration

In [ ]:
# environment
env_name = 'op-env'

try:
    env = ml_client.environments.get(name=env_name, version="2")
    print("Found existing environment.")

except Exception as ex:
    #Print the error message
    print(ex)
    
    print("Creating new enviroment")
    env_docker_conda = Environment(
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        conda_file="../src/env.yml",
        name=env_name,
        description="Environment created from a Docker image plus Conda environment.",
    )
    env = ml_client.environments.create_or_update(env_docker_conda)

## 1.1.5 Prepare Example Data

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

order_path = "../sample_data/order_large.csv"
distances_path = "../sample_data/distance.csv"
# set the version number of the data asset
v1 = "initial"

order_data = Data(
    name="order-data",
    version=v1,
    description="Example order data",
    path=order_path,
    type=AssetTypes.URI_FILE,
)

distances_data = Data(
    name="distances-data",
    version=v1,
    description="Example distance data",
    path=distances_path,
    type=AssetTypes.URI_FILE,
)

## create order data asset if it doesn't already exist:
try:
    order_data_asset = ml_client.data.get(name=order_data.name, version=order_data.version)
    print(
        f"Data asset already exists. Name: {order_data.name}, version: {order_data.version}"
    )
except:
    ml_client.data.create_or_update(order_data)
    print(f"Data asset created. Name: {order_data.name}, version: {order_data.version}")

## create distances data asset if it doesn't already exist:
try:
    distances_data_asset = ml_client.data.get(name=distances_data.name, version=distances_data.version)
    print(
        f"Data asset already exists. Name: {distances_data.name}, version: {distances_data.version}"
    )
except:
    ml_client.data.create_or_update(distances_data)
    print(f"Data asset created. Name: {distances_data.name}, version: {distances_data.version}")


# 1.2 Set up Azure ML Pipeline

This section contains the main logic of the optimization pipeline.

## 1.2.1 Reduce the search space of the problem

The first step is to reduce the search space by assigning some of the orders based on heuristic. The detailed logic is implemented in the reduce.py. In general, if we use heuristic propoerly, we can achieve a good trade-off between result optimality and running time.

In [8]:
src_dir = '../src'

In [9]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

reduce_component = command(
    name="reduce_step",
    display_name="Reduce the problem space",
    description="Read the model input, create partial assignment by heuristic",
    inputs={
        "model_input": Input(type="uri_file"),
        "distance": Input(type="uri_file"),
    },
    outputs=dict(
        model_result_partial=Output(type="uri_folder", mode="rw_mount"),
        model_input_reduced=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=src_dir,
    command="""python reduce.py \
            --model_input ${{inputs.model_input}} --distance ${{inputs.distance}} \
            --model_result_partial ${{outputs.model_result_partial}} --model_input_reduced ${{outputs.model_input_reduced}} \
            """,
    environment=f"{env.name}:{env.version}",
)

# 1.2.2 Partition the problem

For large scale optimization problem, the problem space is just so big to solve practically. A commonly used idea is to partition the big problem into many smaller problems. Then solve each smaller problem individually and combine all the results as the final result. In some cases, the partition may not affect the result optimality, for example, in the route optimization problem, we can partition the orders by the delivery sources. In other cases, there will be trade-off between result optimality and running time when partitioning is applied. 

In [10]:
partition_component = command(
    name="partition_step",
    display_name="Partition the big problem to many small problems",
    description="Read the reduced model input, partition the problem based on some heuristic",
    inputs={
        "model_input_reduced": Input(type="uri_folder"),
        "distance": Input(type="uri_file"),
    },
    outputs=dict(
        model_input_list=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=src_dir,
    command="""python partition.py \
            --model_input_reduced ${{inputs.model_input_reduced}} --distance ${{inputs.distance}} \
            --model_input_list ${{outputs.model_input_list}} \
            """,
    environment=f"{env.name}:{env.version}",
)

## 1.2.3 Solve individual problem

After the problem is partitioned, we can solve each individul one by using whatever optimization solver. The optimization solver itself may leverage multi-process to speed up the search of result. This level of parallelism is totally controlled by the solver but not our Azure ML pipeline.

In [11]:
# parallel task to process file data
solve_component = parallel_run_function(
    name="parallel_solver",
    display_name="Solve the small problems in parallel",
    description="parallel component for problem solver",
    inputs=dict(
        model_input_list=Input(
            type=AssetTypes.URI_FOLDER,
            description="The data to be split and scored in parallel",
        ),
        distance=Input(type=AssetTypes.URI_FOLDER, description='The distance file used by the solver.')
    ),
    outputs=dict(model_result_list=Output(type=AssetTypes.MLTABLE)),
    input_data="${{inputs.model_input_list}}",
    instance_count=10,
    max_concurrency_per_instance=1,
    mini_batch_size="1",
    mini_batch_error_threshold=1,
    retry_settings=dict(max_retries=2, timeout=60),
    logging_level="DEBUG",
    task=RunFunction(
        code=src_dir,
        entry_script="solve.py",
        program_arguments="--distance ${{inputs.distance}}",
        environment=f"{env.name}:{env.version}",
        append_row_to="${{outputs.model_result_list}}",
    ),
)

## 1.2.4 Merge the results

Once all the smaller problems are solved, we can combine the result as the final one. There could be chance to further optimize the result in this step in the case the previous partitioning will affect the global optimal. For example, one may combine two packages into the same truck from two seperated result if the combined one is more cost-efficient. 

In [12]:

merge_component = command(
    name="merge_step",
    display_name="Merge the result of the small problems",
    description="Merge the intermediate result as the final result",
    inputs={
        "model_input": Input(type="uri_file"),
        "distance": Input(type="uri_file"),
        "model_result_partial": Input(type="uri_folder"),
        "model_result_list": Input(type="uri_folder"),
    },
    outputs=dict(
        model_result_final=Output(type="uri_folder", mode="rw_mount"),
    ),
    # The source folder of the component
    code=src_dir,
    command="""python merge.py \
            --model_input ${{inputs.model_input}} --distance ${{inputs.distance}} \
            --model_result_partial ${{inputs.model_result_partial}} --model_result_list ${{inputs.model_result_list}} \
            --model_result_final ${{outputs.model_result_final}} \
            """,
    environment=f"{env.name}:{env.version}",
)

## 1.2.5 Run the Pipeline

Finally, we chained all steps into a single Azure ML pipeline and submit it to run.

In [13]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute=cpu_compute_target, 
    description="E2E route optimization pipeline",
)
def route_optimization_pipeline(
    pipeline_job_model_input,
    pipeline_job_distance,
):
    # initialize the reduce step
    reduce_job = reduce_component(
        model_input=pipeline_job_model_input,
        distance=pipeline_job_distance
    )

    # initialize the partition step
    partition_job = partition_component(
        model_input_reduced=reduce_job.outputs.model_input_reduced,  # note: using outputs from previous step
        distance=pipeline_job_distance,  
    )

    # initialize the solve step
    solve_job = solve_component(
        model_input_list=partition_job.outputs.model_input_list,  # note: using outputs from previous step
        distance=pipeline_job_distance,  
    )

    # initialize the merge step
    merge_job = merge_component(
        model_input=pipeline_job_model_input,  # note: using outputs from previous step
        distance=pipeline_job_distance,  
        model_result_partial=reduce_job.outputs.model_result_partial, # note: using outputs from previous step
        model_result_list=solve_job.outputs.model_result_list, # note: using outputs from previous step
    )

    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    # pipeline_job_model_result_final
    return {
        "pipeline_job_model_result_final": merge_job.outputs.model_result_final,
    }

In [14]:
# Let's instantiate the pipeline with the parameters of our choice
pipeline = route_optimization_pipeline(
    pipeline_job_model_input=Input(type="uri_file", path=order_data_asset.id, mode=InputOutputModes.RO_MOUNT),
    pipeline_job_distance=Input(type="uri_file", path=distances_data_asset.id, mode=InputOutputModes.RO_MOUNT),
)

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name="route_optimization_demo",
)
ml_client.jobs.stream(pipeline_job.name)

# 1.3 Check the Model Result

In [ ]:
sample_schedule_path = '../tmp'

# output = ml_client.jobs.download(name=pipeline_job.name, download_path=sample_schedule_path, all=True)
# Download specific output
output = ml_client.jobs.download(name=pipeline_job.name, download_path=sample_schedule_path, output_name='pipeline_job_model_result_final')

In [17]:
import pandas as pd

model_output = pd.read_csv('../tmp/named-outputs/pipeline_job_model_result_final/schedule.csv')

In [ ]:
model_output.head()